### Develop a program for executing simulation event by event

##### Installation of library

In [1]:
pip install simpy==4.0.1

Note: you may need to restart the kernel to use updated packages.


In [2]:
import simpy
import random
import statistics

In [3]:
### Inital Variable 
wait_times = []

In [4]:
### A class "Theater" contain event for simulation 
class Theater(object):
    def __init__(self, env, num_cashiers, num_servers, num_ushers):
        self.env = env
        self.cashier =simpy.Resource(env, num_cashiers)
        self.server = simpy.Resource(env, num_servers)
        self.usher = simpy.Resource(env, num_ushers)

    def purchase_ticket(self, moviegoer):
        yield self.env.timeout(random.randint(1, 3))

    def check_ticket(self, moviegoer):
        yield self.env.timeout(3 / 60)

    def sell_food(self, moviegoer):
        yield self.env.timeout(random.randint(1, 5))

In [5]:
def go_to_movies(env, moviegoer, theater):
    # Movie goer arrives at the theater
    arrival_time = env.now

    with theater.cashier.request() as request:
        yield request
        yield env.process(theater.purchase_ticket(moviegoer))

    with theater.usher.request() as request:
        yield request
        yield env.process(theater.check_ticket(moviegoer))

    if random.choice([True, False]):
        with theater.server.request() as request:
            yield request
            yield env.process(theater.sell_food(moviegoer))

    # Moviegoer heads into the theater
    wait_times.append(env.now - arrival_time)

In [6]:
def run_theater(env, num_cashiers, num_servers, num_ushers):
    theater = Theater(env, num_cashiers, num_servers, num_ushers)

    for moviegoer in range(3):
        env.process(go_to_movies(env, moviegoer, theater))

    while True:
        yield env.timeout(0.20)  # Wait a bit before generating a new person

        moviegoer += 1
        env.process(go_to_movies(env, moviegoer, theater))

In [7]:
def get_average_wait_time(wait_times):
    average_wait = statistics.mean(wait_times)
    # Pretty print the results
    minutes, frac_minutes = divmod(average_wait, 1)
    seconds = frac_minutes * 60
    return round(minutes), round(seconds)

In [8]:
def get_user_input():
    num_cashiers = input("Input number of cashiers working : ")
    num_servers = input("Input number of servers working : ")
    num_users = input("Input number of users working : ")
    params = [num_cashiers, num_servers, num_users]
    if all(str(i).isdigit() for i in params):  # Check input is valid
        params = [int(x) for x in params]
    else:
        print("Could not parse input, Simulation will use default values: \n1 cashier, 1 server, 1 user.")
        params = [1, 1, 1]
    return params

In [9]:
def main():
    # Setup
    random.seed(42)
    num_cashiers, num_servers, num_ushers = get_user_input()

    # Run the simulation
    env = simpy.Environment()
    env.process(run_theater(env, num_cashiers, num_servers, num_ushers))
    env.run(until=90)

    # View the results
    mins, secs = get_average_wait_time(wait_times)
    print("Running simulation...",
        f"\nThe average wait time is {mins} minutes and {secs} seconds.")

In [10]:
## Run simulation 
if __name__ == "__main__":
    main()

Input number of cashiers working : 10
Input number of servers working : 15
Input number of users working : 14
Running simulation... 
The average wait time is 3 minutes and 47 seconds.
